In [1]:
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
import argparse

In [2]:
import pandas as pd
import numpy as np

In [17]:
x_features = np.load('./BERT_Features/Video_Games_features_BERT.npy')

In [18]:
labels = pd.read_csv('./Video_Games/polarity.txt',header=None)

In [19]:
labels.shape

(50000, 1)

In [20]:
y = []
for i in labels[0]:
    if i=='positive':
        y.append(1)
    else:
        y.append(0)

In [21]:
y = np.array(y)

In [22]:
print(x_features.shape)
print(y.shape)

(50000, 768)
(50000,)


In [23]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import classification_report,confusion_matrix
x_train,x_test,y_train,y_test = train_test_split(x_features,y,test_size=0.2,random_state=47)

In [24]:

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model", dest="model", type=str, default=None, help="gcfoest Net Model File")
    args = parser.parse_args()
    return args


def get_toy_config():
    config = {}
    ca_config = {}
    ca_config["random_state"] = 0
    ca_config["max_layers"] = 100
    ca_config["early_stopping_rounds"] = 3
    ca_config["n_classes"] = 2
    ca_config["estimators"] = []
    ca_config["estimators"].append(
            {"n_folds": 5, "type": "LGBClassifier", "n_estimators": 10, "max_depth": 5,
             "objective": "binary", "silent": True, "nthread": 5, "learning_rate": 0.1} )
    ca_config["estimators"].append({"n_folds": 5, "type": "RandomForestClassifier", "n_estimators": 10, "max_depth": None, "n_jobs": -1})
    ca_config["estimators"].append({"n_folds": 5, "type": "ExtraTreesClassifier", "n_estimators": 10, "max_depth": None, "n_jobs": -1})
    ca_config["estimators"].append({"n_folds": 5, "type": "LogisticRegression"}),
    #ca_config["estimators"].append({"n_folds": 5, "type": "GradientBoostingClassifier"})
    config["cascade"] = ca_config
    return config

In [25]:
config = get_toy_config()
gc = GCForest(config)

In [26]:
gc.config

{'cascade': {'random_state': 0,
  'max_layers': 100,
  'early_stopping_rounds': 3,
  'n_classes': 2,
  'estimators': [{'n_folds': 5,
    'type': 'LGBClassifier',
    'n_estimators': 10,
    'max_depth': 5,
    'objective': 'binary',
    'silent': True,
    'nthread': 5,
    'learning_rate': 0.1},
   {'n_folds': 5,
    'type': 'RandomForestClassifier',
    'n_estimators': 10,
    'max_depth': None,
    'n_jobs': -1},
   {'n_folds': 5,
    'type': 'ExtraTreesClassifier',
    'n_estimators': 10,
    'max_depth': None,
    'n_jobs': -1},
   {'n_folds': 5, 'type': 'LogisticRegression'}]}}

In [27]:
X_train_enc = gc.fit_transform(x_train, y_train,X_test=x_test, y_test=y_test)
#y_pred = gc.predict(x_test)

[ 2019-03-28 15:31:07,990][cascade_classifier.fit_transform] X_groups_train.shape=[(40000, 768)],y_train.shape=(40000,),X_groups_test.shape=[(10000, 768)],y_test.shape=(10000,)
[ 2019-03-28 15:31:08,043][cascade_classifier.fit_transform] group_dims=[768]
[ 2019-03-28 15:31:08,044][cascade_classifier.fit_transform] group_starts=[0]
[ 2019-03-28 15:31:08,045][cascade_classifier.fit_transform] group_ends=[768]
[ 2019-03-28 15:31:08,045][cascade_classifier.fit_transform] X_train.shape=(40000, 768),X_test.shape=(10000, 768)
[ 2019-03-28 15:31:08,098][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(40000, 768), X_cur_test.shape=(10000, 768)
[ 2019-03-28 15:31:09,876][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 5_folds.train_0.predict)=53.17%
[ 2019-03-28 15:31:11,726][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 5_folds.train_1.predict)=52.56%
[ 2019-03-28 15:31:13,530][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 -

[ 2019-03-28 15:34:02,267][kfold_wrapper.log_eval_metrics] Accuracy(layer_1 - estimator_3 - 5_folds.train_0.predict)=58.17%
/home/mounika/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[ 2019-03-28 15:34:19,404][kfold_wrapper.log_eval_metrics] Accuracy(layer_1 - estimator_3 - 5_folds.train_1.predict)=57.53%
/home/mounika/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[ 2019-03-28 15:34:40,910][kfold_wrapper.log_eval_metrics] Accuracy(layer_1 - estimator_3 - 5_folds.train_2.predict)=58.19%
/home/mounika/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.


[ 2019-03-28 15:37:50,505][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 5_folds.train_3.predict)=52.32%
[ 2019-03-28 15:37:52,826][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 5_folds.train_4.predict)=52.02%
[ 2019-03-28 15:37:52,943][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 5_folds.train_cv.predict)=52.83%
[ 2019-03-28 15:37:52,946][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_1 - 5_folds.test.predict)=55.15%
[ 2019-03-28 15:37:54,363][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_2 - 5_folds.train_0.predict)=53.29%
[ 2019-03-28 15:37:55,797][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_2 - 5_folds.train_1.predict)=53.52%
[ 2019-03-28 15:37:57,211][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_2 - 5_folds.train_2.predict)=52.51%
[ 2019-03-28 15:37:58,629][kfold_wrapper.log_eval_metrics] Accuracy(layer_3 - estimator_2 - 5_folds.train_3.predict)=53.26%
[ 2019-03-

[ 2019-03-28 15:41:33,379][cascade_classifier.calc_accuracy] Accuracy(layer_4 - train.classifier_average)=57.19%
[ 2019-03-28 15:41:33,379][cascade_classifier.calc_accuracy] Accuracy(layer_4 - test.classifier_average)=56.27%
[ 2019-03-28 15:41:33,454][cascade_classifier.fit_transform] [layer=5] look_indexs=[0], X_cur_train.shape=(40000, 776), X_cur_test.shape=(10000, 776)
[ 2019-03-28 15:41:35,217][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 5_folds.train_0.predict)=58.24%
[ 2019-03-28 15:41:36,962][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 5_folds.train_1.predict)=58.11%
[ 2019-03-28 15:41:38,702][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 5_folds.train_2.predict)=58.49%
[ 2019-03-28 15:41:40,467][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 5_folds.train_3.predict)=58.83%
[ 2019-03-28 15:41:42,311][kfold_wrapper.log_eval_metrics] Accuracy(layer_5 - estimator_0 - 5_folds.train_4.predict)=58.97%
[ 201

/home/mounika/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[ 2019-03-28 15:44:49,882][kfold_wrapper.log_eval_metrics] Accuracy(layer_6 - estimator_3 - 5_folds.train_1.predict)=57.49%
/home/mounika/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[ 2019-03-28 15:45:10,618][kfold_wrapper.log_eval_metrics] Accuracy(layer_6 - estimator_3 - 5_folds.train_2.predict)=56.71%
/home/mounika/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[ 2019-03-28 15:45:29,338][kfold_wrapper.log_eval_metrics] Accuracy(layer_6 - estimator_3 - 5_folds.train_3

[ 2019-03-28 15:48:19,389][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_1 - 5_folds.train_4.predict)=52.91%
[ 2019-03-28 15:48:19,505][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_1 - 5_folds.train_cv.predict)=53.95%
[ 2019-03-28 15:48:19,508][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_1 - 5_folds.test.predict)=55.40%
[ 2019-03-28 15:48:20,725][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_2 - 5_folds.train_0.predict)=53.33%
[ 2019-03-28 15:48:22,123][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_2 - 5_folds.train_1.predict)=52.58%
[ 2019-03-28 15:48:23,454][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_2 - 5_folds.train_2.predict)=53.73%
[ 2019-03-28 15:48:24,873][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_2 - 5_folds.train_3.predict)=52.74%
[ 2019-03-28 15:48:26,189][kfold_wrapper.log_eval_metrics] Accuracy(layer_8 - estimator_2 - 5_folds.train_4.predict)=53.26%
[ 2019-03-

In [28]:
y_pred = gc.predict(x_test)

[ 2019-03-28 15:50:08,013][cascade_classifier.transform] X_groups_test.shape=[(10000, 768)]
[ 2019-03-28 15:50:08,019][cascade_classifier.transform] group_dims=[768]
[ 2019-03-28 15:50:08,020][cascade_classifier.transform] X_test.shape=(10000, 768)
[ 2019-03-28 15:50:08,025][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(10000, 768)
[ 2019-03-28 15:50:09,457][cascade_classifier.transform] [layer=1] look_indexs=[0], X_cur_test.shape=(10000, 776)
[ 2019-03-28 15:50:10,918][cascade_classifier.transform] [layer=2] look_indexs=[0], X_cur_test.shape=(10000, 776)
[ 2019-03-28 15:50:12,366][cascade_classifier.transform] [layer=3] look_indexs=[0], X_cur_test.shape=(10000, 776)
[ 2019-03-28 15:50:13,820][cascade_classifier.transform] [layer=4] look_indexs=[0], X_cur_test.shape=(10000, 776)
[ 2019-03-28 15:50:15,227][cascade_classifier.transform] [layer=5] look_indexs=[0], X_cur_test.shape=(10000, 776)


In [29]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.57      0.55      0.56      5003
           1       0.56      0.57      0.57      4997

   micro avg       0.56      0.56      0.56     10000
   macro avg       0.56      0.56      0.56     10000
weighted avg       0.56      0.56      0.56     10000



In [30]:
import pickle
pickle.dump(gc,open('/home/mounika/Desktop/Word-Embeddings/dranziera_protocol/GC_BERT/gc_bert_Video_Games','wb'))